In [13]:
import numpy as np
#from imageio import imread, imwrite
import cv2
# from matplotlib,pyplot as plt # not working for me

## Naive Method: Increase blue channel

In [14]:
image = cv2.imread('grainger.jpg')

arr = image * np.array([0.5, 0.2, 0.1])
arr2 = (255 * arr/arr.max()).astype(np.uint8)

gamma = 2
gamma_img = np.array(255 * (arr2/255) ** gamma, dtype="uint8")
cv2.imwrite("night_final.png", gamma_img)

True

## Advanced Method: Remove illumination, decrease exposure, relight with night illuminants, and add noise

### Remove illumination

Stacked RGGB Bayer image:
$$ \mathbf{I}_{day} \in R^{\frac{H}{2} \times \frac{W}{2} \times 4} $$

In [15]:
def bggr_to_bgr(bggr):
    bgr_img = np.zeros((bggr.shape[0], bggr.shape[1], 3))
    bgr_img[:,:,0] = bggr[:,:,0]
    bgr_img[:,:,1] = (bggr[:,:,1] + bggr[:,:,2]) / 2
    bgr_img[:,:,2] = bggr[:,:,3]
    return bgr_img


In [16]:
img = cv2.imread('grainger.jpg')
n_channels = 4
I_day = np.zeros((img.shape[0], img.shape[1], n_channels))
I_day[:,:,0] = img[:,:,0]
I_day[:,:,1] = img[:,:,1]
I_day[:,:,2] = img[:,:,1].copy()
I_day[:,:,3] = img[:,:,2]
print(I_day.shape)
print(I_day)


(1200, 1800, 4)
[[[240. 226. 226. 220.]
  [240. 226. 226. 220.]
  [240. 226. 226. 220.]
  ...
  [246. 246. 246. 246.]
  [246. 246. 246. 246.]
  [246. 246. 246. 246.]]

 [[240. 226. 226. 220.]
  [240. 226. 226. 220.]
  [240. 226. 226. 220.]
  ...
  [246. 246. 246. 246.]
  [246. 246. 246. 246.]
  [246. 246. 246. 246.]]

 [[240. 226. 226. 220.]
  [240. 226. 226. 220.]
  [240. 226. 226. 220.]
  ...
  [246. 246. 246. 246.]
  [246. 246. 246. 246.]
  [246. 246. 246. 246.]]

 ...

 [[ 51.  98.  98.  89.]
  [ 50.  97.  97.  88.]
  [ 34.  81.  81.  72.]
  ...
  [ 50.  96.  96.  83.]
  [ 40.  88.  88.  76.]
  [ 24.  73.  73.  59.]]

 [[ 49.  96.  96.  87.]
  [ 31.  78.  78.  69.]
  [ 35.  82.  82.  73.]
  ...
  [ 38.  85.  85.  69.]
  [ 34.  83.  83.  69.]
  [ 31.  80.  80.  64.]]

 [[ 52.  99.  99.  90.]
  [ 30.  77.  77.  68.]
  [ 49.  96.  96.  87.]
  ...
  [ 28.  75.  75.  59.]
  [ 28.  75.  75.  59.]
  [ 37.  86.  86.  70.]]]


In [17]:
# The process begins with a "minimally processed Bayer image recorded by the camera sensor".
# We could synthesize the effect of a raw Bayer image with the code below,
# then interpolate the pixel values (or downsample, but I suppose it's arbitrary). This is why I think their I_day is H/2 x W/2
# OR this may all be useless and we should just contain with the normal rgb image turned into rggb

# Created with the help of ChatGPT.
# def apply_bayer_pattern(image):
#     print(image.shape)
#     # Create a blank Bayer pattern image with the same dimensions as the RGB image
#     # bayer_image = np.zeros(image.shape * 2)

#     new_height, new_width = image.shape[0] * 2, image.shape[1] * 2
#     bayer = np.zeros((new_height, new_width))
#     bayer[0::2, 0::2] = image[:, :, 0]
#     bayer[0::2, 1::2] = image[:, :, 1]
#     bayer[1::2, 0::2] = image[:, :, 1]
#     bayer[1::2, 1::2] = image[:, :, 2]

#     return bayer

# I_day = apply_bayer_pattern(img)
# cv2.imwrite('bayer_image.jpg', I_day)


Normalized image:
$$ \mathbf{I}_n = (\mathbf{I}_{day}-b_l) / (w_l-b_l) $$

In [18]:
I_day = img
print(I_day.shape)
# wl = np.mean(np.max(img, axis=(0,1)))
# bl = np.mean(np.min(img, axis=(0,1)))
wl = np.max(np.sum(I_day, axis=2))
bl = np.min(np.sum(I_day, axis=2))

I_n = (I_day - bl) / (wl - bl)
print(I_n)

# plt.figure()
# plt.imshow(I_n)
cv2.imwrite("I_n.png", I_n * 255)
print(I_n)

True

White-balanced image:
$$ \mathbf{I}_w = \mathbf{I}_n\mathbf{L}_{day} $$
where
$$ {L}_{day} = diag(\frac{1}{b}, \frac{1}{g}, \frac{1}{g}, \frac{1}{r}) $$

In [19]:
print(I_n.shape)

for c in range(I_n.shape[-1]):
    pass

# https://chat.openai.com/share/f90475c2-2156-42c5-afba-2b8a22fee9f1

L_day = np.identity(I_n.shape[-1])
I_w = I_n @ L_day

I_w = np.clip((I_n*1.0 / I_n.mean(axis=(0,1))), 0, 1)
cv2.imwrite("I_w.png", I_w * 255)

# We might just have to come up with some values for r, g, b to be the average shift of intensities to color balance to neutral
# Or create a simple algorithm

IndexError: index 3 is out of bounds for axis 2 with size 3

### Lowering brightness

`D`: dictionary for the normalized mean intensity value of each Bayer image
(could just make list with index as key for the image)

In [ ]:
# I believe bayer_images should have a few nighttime images, which would make D have normalized means of random nighttime photos
bayer_images = np.array([I_day])
print(bayer_images.shape)

# contains normalized mean intensity values of each Bayer image
D = np.array([np.mean(img) / np.max(img) for img in bayer_images])

global_scale_factor_d = 0.3 #np.random.choice(D)
print(global_scale_factor_d)

I_e = I_w * global_scale_factor_d
cv2.imwrite("I_e.png", I_e * 255)


(1, 1200, 1800, 4)
0.5865332874909223


### Adding illuminants

`L`: dictionary for nighttime illuminants

In [ ]:
import numpy as np

def generate_2d_gaussian_array(shape, sigma=1.0, center=None):
    """
    Generate a 2-D Gaussian array.

    Parameters:
        shape (tuple): Shape of the output array.
        sigma (float): Standard deviation of the Gaussian filter.
        center (tuple): Center of the Gaussian filter. Default is None, which sets the center to the center of the array.

    Returns:
        numpy.ndarray: 2-D Gaussian array with the specified shape.
    """
    if center is None:
        center = (shape[0] // 2, shape[1] // 2)

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
    x -= center[0]
    y -= center[1]

    exponent = -(x**2 + y**2) / (2 * sigma**2)
    gaussian_array = np.exp(exponent) / (2 * np.pi * sigma**2)

    return gaussian_array

(5, 2)
[7.12364326e-06 8.25754705e-06] (2,)
[[ 2.96163521e-11 -1.12806656e-12]
 [-1.12806656e-12  4.57012992e-11]] (2, 2)


In [ ]:
i_r = np.zeros(I_e.shape)

night_time_illuminant = [[.5, .3, .3], [0, 1, 1], [0, 1, 1]]


M = generate_2d_gaussian_array((1800,1200),sigma=(500))

i_r[:,:,0] += (I_e * night_time_illuminant[0])[:,:,0] * 2 
i_r[:,:,1] += (I_e * night_time_illuminant[0])[:,:,1] * 2
i_r[:,:,2] += (I_e * night_time_illuminant[0])[:,:,2] * 2


M = generate_2d_gaussian_array((1800,1200),sigma=(150), center=(200,600))
i_r[:,:,0] += (I_e * night_time_illuminant[1])[:,:,0]* M * 3e4
i_r[:,:,1] += (I_e * night_time_illuminant[1])[:,:,1] * M * 3e4
i_r[:,:,2] += (I_e * night_time_illuminant[1])[:,:,2] * M * 3e4

M = generate_2d_gaussian_array((1800,1200),sigma=(150), center=(1600,600))
i_r[:,:,0] += (I_e * night_time_illuminant[2])[:,:,0]* M * 3e4
i_r[:,:,1] += (I_e * night_time_illuminant[2])[:,:,1] * M * 3e4
i_r[:,:,2] += (I_e * night_time_illuminant[2])[:,:,2] * M * 3e4

M = generate_2d_gaussian_array((1800,1200),sigma=(150), center=(900,600))
i_r[:,:,0] += (I_e * night_time_illuminant[2])[:,:,0]* M * 3e4
i_r[:,:,1] += (I_e * night_time_illuminant[2])[:,:,1] * M * 3e4
i_r[:,:,2] += (I_e * night_time_illuminant[2])[:,:,2] * M * 3e4
print(M)
print(np.max(M))
print(np.min(M))
#print((I_e * night_time_illuminant)[:,:,2] )
noise = np.random.normal(0,5, i_r.shape)
cv2.imwrite("I_r.png", (i_r * 255) + noise)

In [ ]:
i_night = i_r * (wl - bl) + bl
cv2.imwrite("I_night.png", i_night)

ValueError: operands could not be broadcast together with shapes (1200,1800,4) (3,) 

In [ ]:
i_night = i_r * (wl - bl) + bl
cv2.imwrite("I_night.png", i_night)

### Adding noise

### Final Result